<a href="https://colab.research.google.com/github/nikita-resh/StyleGAN-NADA/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install Ninja

import torch
import matplotlib.pyplot as plt
import gdown
import clip
import gc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-n32blp3j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-n32blp3j
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/rosinality/stylegan2-pytorch.git

Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 395, done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 395 (from 1)
Receiving objects: 100% (395/395), 122.51 MiB | 25.23 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [3]:
%cd stylegan2-pytorch

/content/stylegan2-pytorch


In [4]:
from model import Generator

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [5]:
file_path = "/content/stylegan2-pytorch/lpips/__init__.py"

# Откроем файл и прочитаем его содержимое
with open(file_path, "r") as file:
    lines = file.readlines()

# Заменим нужную строку
with open(file_path, "w") as file:
    for line in lines:
        # Если в строке есть "compare_ssim", заменим на "structural_similarity"
        if "from skimage.measure import compare_ssim" in line:
            file.write("from skimage.metrics import structural_similarity\n")
        else:
            file.write(line)

print("Замена выполнена успешно!")

Замена выполнена успешно!


In [6]:
# Зададим девайс: cuda или cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# URL for the Google Drive file
url = "https://drive.google.com/uc?id=1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT"

# Download the file
gdown.download(url, "stylegan2-ffhq-config-f.pt", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT
From (redirected): https://drive.google.com/uc?id=1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT&confirm=t&uuid=fe6caed8-e102-4589-abc8-ebdcafb3811f
To: /content/stylegan2-pytorch/stylegan2-ffhq-config-f.pt
100%|██████████| 381M/381M [00:01<00:00, 242MB/s]


'stylegan2-ffhq-config-f.pt'

### CLIP Loss

We want the model to generate images that align with the properties described in the text prompt. To achieve this, we use CLIP loss to encourage the model to generate images that are closer to the text description in the CLIP embedding space.

The CLIP loss is computed as follows:

$$
D_{\text{CLIP}} = 1 - \frac{\langle E_T(G(w)) \cdot E_T(t) \rangle }{\|E_T(G(w))\| \cdot \|E_T(t)\|}
$$

**Where:**

- $E_T$: Text and image encoder function (e.g., CLIP's shared encoder for both text and image).
- $t$: Text description (prompt).
- $G(w)$: Image generated by the generator using the latent vector $w$.
- $D_{\text{CLIP}}$: Cosine distance between the text embedding and the image embedding in the CLIP space.

**Intuition:** The CLIP loss encourages the generated image $G(w)$ to be semantically aligned with the provided text prompt $t$. If the image and text embeddings are close in the CLIP space, the loss will be small, helping the model generate images that reflect the described properties.


In [8]:
class CLIPLoss(torch.nn.Module):
    def __init__(self, stylegan_size=1024):
        super(CLIPLoss, self).__init__()

        self.model, self.preprocess = clip.load("ViT-B/32", device=device)

        # clip only supports images 224*224.
        # we need to resize image with min information loss possible. therefore we firstly apply upsamling, and then pooling.
        # scale_factor=7 allows not to deal with decimals
        self.upsample = torch.nn.Upsample(scale_factor=7)
        self.avg_pool = torch.nn.AvgPool2d(kernel_size=(1024 * 7 // 224))

    def forward(self, prompt, image):
        # text preprocessing
        tokenized_prompt = clip.tokenize([prompt]).to(device)
        # image preprocessing
        preprocessed_image = self.avg_pool(self.upsample(image))
        # encoding to CLIP space
        text_features = self.model.encode_text(tokenized_prompt)  # Remove .detach()
        image_features = self.model.encode_image(preprocessed_image)  # Remove .detach()

        # normalize embeddings
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)

        similarity = (image_features * text_features).sum(dim=1)
        loss = 1 - similarity.mean()

        return loss

### CLIP Directional loss

We want to only change properties described in our text prompt. We use CLIP loss to panish model for changing other properties.

The directional loss is computed as follows:

$$
\Delta T = E_T(t_{\text{target}}) - E_T(t_{\text{source}})
$$

$$
\Delta I = E_I(G_{\text{train}}(w)) - E_I(G_{\text{frozen}}(w))
$$

$$
L_{\text{direction}} = 1 - \frac{\Delta I \cdot \Delta T}{|\Delta I| \cdot |\Delta T|}
$$

**Where:**

- $E_T$: Text encoder function (e.g., CLIP text encoder).
- $E_I$: Image encoder function (e.g., CLIP image encoder).
- $t_{\text{target}}$: Target text description.
- $t_{\text{source}}$: Source text description.
- $G_{\text{train}}$: Trainable generator (e.g., StyleGAN during training).
- $G_{\text{frozen}}$: Frozen generator (pre-trained StyleGAN, unmodified during training).
- $w$: Latent vector input to the generator.

**Intuition:** If the image changes $\Delta I$ don’t align with the text changes $\Delta T$, the loss increases, encouraging the generator to better follow the text guidance during optimization.


In [9]:
class CLIPDirectionalLoss(torch.nn.Module):
    def __init__(self, stylegan_size=1024):
        super(CLIPDirectionalLoss, self).__init__()

        self.model, self.preprocess = clip.load("ViT-B/32", device=device)

        # clip only supports images 224*224.
        # we need to resize image with min information loss possible. therefore we firstly apply upsamling, and then pooling.
        # scale_factor=7 allows not to deal with decimals
        self.upsample = torch.nn.Upsample(scale_factor=7)
        self.avg_pool = torch.nn.AvgPool2d(kernel_size=(1024 * 7 // 224))

    def forward(
        self,
        frozen_generator_image,
        trainable_generator_image,
        source_prompt,
        target_prompt,
    ):
        # text preprocessing
        tokenized_source_prompt = clip.tokenize([source_prompt]).to(device)
        tokenized_target_prompt = clip.tokenize([target_prompt]).to(device)
        # image preprocessing
        preprocessed_frozen_generator_image = self.avg_pool(
            self.upsample(frozen_generator_image)
        )
        preprocessed_trainable_generator_image = self.avg_pool(
            self.upsample(trainable_generator_image)
        )

        # encoding to CLIP space
        source_prompt_features = self.model.encode_text(tokenized_source_prompt)
        target_prompt_features = self.model.encode_text(tokenized_target_prompt)
        frozen_generator_image_features = self.model.encode_image(
            preprocessed_frozen_generator_image
        )
        trainable_generator_image_features = self.model.encode_image(
            preprocessed_trainable_generator_image
        )

        delta_t = target_prompt_features - source_prompt_features
        delta_i = trainable_generator_image_features - frozen_generator_image_features

        # normalize deltas
        delta_i = delta_i / delta_i.norm(dim=1, keepdim=True)
        delta_t = delta_t / delta_t.norm(dim=1, keepdim=True)

        loss = 1 - (delta_i * delta_t).sum(dim=1).mean()

        return loss

In [10]:
clip_loss = CLIPLoss()
clip_directional_loss = CLIPDirectionalLoss()

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 64.0MiB/s]


In [11]:
# Параметры генератора
size = 1024  # Размер изображения
latent_dim = 512  # Размер латентного пространства
n_mlp = 8  # Количество слоев MLP
channel_multiplier = 2
ckpt = "/content/stylegan2-pytorch/stylegan2-ffhq-config-f.pt"  # Путь к весам StyleGAN2

In [12]:
frozen_generator = Generator(
    size, latent_dim, n_mlp, channel_multiplier=channel_multiplier
).to(device)
trainable_generator = Generator(
    size, latent_dim, n_mlp, channel_multiplier=channel_multiplier
).to(device)

frozen_generator.eval()
trainable_generator.train()

checkpoint = torch.load(ckpt)

frozen_generator.load_state_dict(checkpoint["g_ema"])
trainable_generator.load_state_dict(checkpoint["g_ema"])

<ipython-input-12-0f2224f1e908>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt)


<All keys matched successfully>

In [13]:
def freeze_stylegan_layers(generator, selected_layers=None):
    """
    Freezes specific layers in a StyleGAN generator.

    Args:
        generator: The StyleGAN generator model.
        selected_layers: A list of indices of layers that should remain trainable.
                         If None, only Mapping Network, Affine Transformations, and ToRGB layers are frozen.
    """
    # Always freeze Mapping Network (style), ToRGB layers, and Affine Transformations
    modules_to_freeze = ["style", "to_rgb1"]

    for module in modules_to_freeze:
        if hasattr(generator, module):
            for param in getattr(generator, module).parameters():
                param.requires_grad = False

    # Freeze all ToRGB layers in the ModuleList
    if hasattr(generator, "to_rgbs"):
        for to_rgb_layer in generator.to_rgbs:
            for param in to_rgb_layer.parameters():
                param.requires_grad = False

    # If selected_layers is provided, freeze all other conv layers
    if selected_layers is not None:
        for i, conv in enumerate(generator.convs):
            if i not in selected_layers:  # Freeze layers not in the selected list
                for param in conv.parameters():
                    param.requires_grad = False
            else:
                for param in conv.parameters():
                    param.requires_grad = True

In [14]:
freeze_stylegan_layers(frozen_generator)
freeze_stylegan_layers(trainable_generator)

In [15]:
batch_size = 4  # Number of images to use in each step
num_steps = 61

l2_lambda = 1.0
clip_lambda = 2.5
clip_directional_lambda = 1.2
id_lambda = 0.0005
num_layers = 18


optimizer = torch.optim.Adam(trainable_generator.parameters(), lr=0.01)

source_prompt = "photo"
target_prompt = "zombie"

In [16]:
import gc

In [17]:
def get_k_relevant_layers(generator, k, target_prompt):
    latent_z = torch.randn((batch_size, latent_dim), device=device)

    generator.eval()
    with torch.no_grad():
        latent_w = generator.style(latent_z).detach()
        latent_w_plus = latent_w.unsqueeze(1).repeat(1, num_layers, 1).detach().clone()

    latent_w_plus.requires_grad_(True)  # Указываем requires_grad_ отдельно

    # Сохраняем начальное состояние
    initial_w_plus = latent_w_plus.clone().detach()

    relevant_optimizer = torch.optim.Adam([latent_w_plus], lr=0.01)

    for step in range(10):
        relevant_optimizer.zero_grad()

        image, _ = generator(
            [latent_w_plus], input_is_latent=True, randomize_noise=False
        )
        loss = clip_lambda * clip_loss(target_prompt, image)

        loss.backward()  # `loss.requires_grad = True` не нужен
        relevant_optimizer.step()

        del image, _, loss

    # Вычисляем изменения после оптимизации
    delta_w = (latent_w_plus.detach() - initial_w_plus).abs().mean(dim=(0, 2))

    # Выбираем k слоёв с наибольшими изменениями
    relevant_layers = [i - 1 for i in delta_w.topk(k).indices.tolist()]
    print("relevant_layers", relevant_layers)

    gc.collect()  # Принудительно запускаем garbage collector
    torch.cuda.empty_cache()  # Очищаем кеш GPU

    return relevant_layers

In [ ]:
losses = {"l2": [], "clip": [], "clip_directional": [], "total": []}

# Количество стилевых модулей в генераторе (обычно 18 для StyleGAN2)
for step in range(num_steps):
    optimizer.zero_grad()

    if step % 20 == 0:
        k_relevant_layers = get_k_relevant_layers(trainable_generator, 5, target_prompt)
        freeze_stylegan_layers(trainable_generator, k_relevant_layers)

    # Генерация латентных векторов
    latent_z = torch.randn((batch_size, latent_dim), device=device)

    # Проекция в W+ пространство
    latent_w = frozen_generator.style(latent_z).detach().clone()
    latent_w_plus = latent_w.unsqueeze(1).repeat(
        1, num_layers, 1
    )  # [batch_size, num_layers, latent_dim]

    # Генерация изображений в W+ пространстве
    with torch.no_grad():
        frozen_generator_images, _ = frozen_generator(
            [latent_w_plus], input_is_latent=True, randomize_noise=False
        )
    trainable_generator_images, _ = trainable_generator(
        [latent_w_plus], input_is_latent=True, randomize_noise=False
    )

    # Вычисление потерь
    l2_loss_value = torch.mean(
        (frozen_generator_images - trainable_generator_images) ** 2
    )
    clip_loss_value = clip_loss(target_prompt, trainable_generator_images)
    clip_directional_loss_value = clip_directional_loss(
        frozen_generator_images,
        trainable_generator_images,
        source_prompt,
        target_prompt,
    )

    # Комбинирование потерь
    loss = (
        l2_lambda * l2_loss_value
        + clip_lambda * clip_loss_value
        + clip_directional_lambda * clip_directional_loss_value
    )

    # Логирование потерь
    losses["l2"].append(l2_loss_value.detach().item())
    losses["clip"].append(clip_loss_value.detach().item())
    losses["clip_directional"].append(clip_directional_loss_value.detach().item())
    losses["total"].append(loss.detach().item())

    # Обратное распространение ошибки и обновление весов
    loss.backward()
    optimizer.step()

    # Visualization and logging
    if step % 20 == 0 or step == num_steps - 1:
        print(f"Step [{step}/{num_steps}], Loss: {losses['total'][-1]}")

        # Create subplots to visualize both frozen and trainable images side-by-side
        fig, axs = plt.subplots(
            2, batch_size, figsize=(batch_size * 4, 8)
        )  # 2 rows, batch_size columns

        # If batch_size = 1, axs will not be iterable, so wrap it in a list
        if batch_size == 1:
            axs = [[axs[0]], [axs[1]]]

        for i in range(batch_size):
            # Display frozen_generator_images (row 0)
            axs[0][i].imshow(
                (
                    frozen_generator_images.cpu()
                    .detach()[i]
                    .clip(-1, 1)
                    .permute(1, 2, 0)
                    + 1
                )
                / 2
            )
            axs[0][i].set_title(f"Frozen Image {i+1}")
            axs[0][i].axis("off")

            # Display trainable_generator_images (row 1)
            axs[1][i].imshow(
                (
                    trainable_generator_images.cpu()
                    .detach()[i]
                    .clip(-1, 1)
                    .permute(1, 2, 0)
                    + 1
                )
                / 2
            )
            axs[1][i].set_title(f"Trainable Image {i+1}")
            axs[1][i].axis("off")

        plt.tight_layout()
        plt.show()

    # Clear cached data to free up memory
    del frozen_generator_images, trainable_generator_images, latent_w, latent_z
    torch.cuda.empty_cache()

In [ ]:
# Steps corresponding to the losses
steps = range(len(losses["total"]))

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
fig.suptitle("Losses Over Steps", fontsize=16)

# Titles and keys for the losses
titles = ["L2 Loss", "CLIP Loss", "CLIP Directional", "Total Loss"]
loss_keys = ["l2", "clip", "clip_directional", "total"]

# Plot each loss
for i, ax in enumerate(axs.flat):
    loss_key = loss_keys[i]
    ax.plot(steps, losses[loss_key], marker="o", label=f"{loss_key.upper()} Loss")
    ax.set_title(titles[i])
    ax.set_xlabel("Step")
    ax.set_ylabel("Loss")
    ax.grid(True)
    ax.legend()

# Adjust layout to fit the title and spacing
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

Inversion


In [ ]:
%cd ..

In [ ]:
!git clone https://github.com/omertov/encoder4editing.git

In [ ]:
%cd encoder4editing

In [ ]:
# Импортируем необходимые библиотеки
from models.psp import pSp  # Импортируем модель pSp
from argparse import Namespace

In [ ]:
# URL for the Google Drive file
url = "https://drive.google.com/uc?id=1cUv_reLE6k3604or78EranS7XzuVMWeO"

# Download the file
gdown.download(url, "e4e_ffhq_encode.pt", quiet=False)

In [ ]:
from torchvision import transforms

In [ ]:
# Определяем преобразования для изображения
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),  # Изменяем размер изображения до 256x256
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
        ),  # Нормализуем значения пикселей
    ]
)

In [ ]:
# Загружаем предобученную модель pSp
model_path = "/content/encoder4editing/e4e_ffhq_encode.pt"  # Путь к весам энкодера
ckpt = torch.load(model_path, map_location="cpu")
opts = ckpt["opts"]

opts["checkpoint_path"] = model_path
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.to(device)
print("Model successfully loaded!")

In [ ]:
# Загружаем и отображаем изображение
from PIL import Image

image_path = "/content/images/mikita.jpeg"  # Путь к изображению
original_image = Image.open(image_path)
original_image = original_image.convert("RGB")  # Преобразуем изображение в RGB формат
original_image

In [ ]:
import os

In [ ]:
# Определяем тип эксперимента
experiment_type = "ffhq_encode"

# Загружаем модель для определения ключевых точек лица - понадобится нам для выравнивания
if experiment_type == "ffhq_encode" and 'shape_predictor_68_face_landmarks.dat' not in os.listdir():
    !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2  # Скачиваем архив с моделью
    !bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2  # Распаковываем архив

# Задаем размер изображения для изменения
resize_dims = (256, 256)


def run_alignment(image_path):
  """Выравнивает лицо на изображении, используя dlib.

  Args:
      image_path: Путь к файлу изображения.

  Returns:
      Выровненное изображение в формате PIL.Image.
  """
  import dlib
  from utils.alignment import align_face  # Функция для выравнивания лица
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor)
  print("Aligned image has shape: {}".format(aligned_image.size))
  return aligned_image  # Возвращаем выровненное изображение

# Выравниваем лицо, если тип эксперимента - кодирование FFHQ
if experiment_type == "ffhq_encode":
  input_image = run_alignment(image_path)
else:
  input_image = original_image


input_image.resize(resize_dims)

In [ ]:
import numpy as np

In [ ]:
# Посмотрим на конечный результат
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asarray(input_image))
axs[0].set_title("Исходное изображение")
axs[0].axis("off")

axs[1].imshow(np.asarray(original_image))
axs[1].set_title("Выровненное изображение")
axs[1].axis("off")

plt.show()

In [ ]:
transformed_image = transform(
    input_image
)  # Применяем преобразования, определенные ранее


# Определяем функцию для отображения результата рядом с исходным
def display_alongside_source_image(result_image, source_image):
    """Отображает результат рядом с исходным изображением.

    Args:
        result_image: Результирующее изображение (PIL.Image).
        source_image: Исходное изображение (PIL.Image).

    Returns:
        Объединенное изображение (PIL.Image).
    """
    res = np.concatenate(
        [
            np.array(source_image.resize(resize_dims)),
            np.array(result_image.resize(resize_dims)),
        ],
        axis=1,
    )
    return Image.fromarray(res)


def run_on_batch(inputs, net):
    """Запускает модель на пакете данных.

    Args:
        inputs: Входные данные (тензор PyTorch).
        net: Модель pSp.

    Returns:
        Сгенерированные изображения и латентные векторы.
    """
    images, latents = net(
        inputs.to("cuda").float(), randomize_noise=False, return_latents=True
    )  # Запускаем модель
    return images, latents

In [ ]:
# Запускаем модель и измеряем время выполнения
import time  # Модуль для работы со временем
from utils.common import tensor2im
import numpy as np

with torch.no_grad():
    tic = time.time()  # Время начала выполнения
    images, latents = run_on_batch(transformed_image.unsqueeze(0), net)
    result_image, latent = (
        images[0],
        latents[0],
    )  # Извлекаем результат и латентный вектор
    toc = time.time()  # Время окончания выполнения
    print("Inference took {:.4f} seconds.".format(toc - tic))

In [ ]:
latent.shape

In [ ]:
# Отображаем результат инверсии
display_alongside_source_image(tensor2im(result_image), input_image)

In [ ]:
trainable_generator.eval()

In [ ]:
trainable_generator_images, _ = trainable_generator(
    [latent], input_is_latent=True, randomize_noise=False
)

In [ ]:
del frozen_generator

In [ ]:
# torch.cuda.empty_cache()

Cleaning


In [ ]:
# torch.cuda.empty_cache()

# # Move tensors to CPU and delete them
# del frozen_generator  # Assuming `model` is your trained model
# del trainable_generator  # Assuming `model` is your trained model
# torch.cuda.empty_cache()

# # Collect garbage to clear references
# gc.collect()